# LORA Fine-tuning

In [21]:
from datasets import *
import datasets

from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    TFAutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)
import tensorflow
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np 

In [22]:

dataset = load_dataset('deepset/prompt-injections')
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 546
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 116
    })
})

### model

In [23]:
model_checkpoint = "distilbert-base-uncased"
model_output = "distilbert-base-uncased-deepset-promptinjection"

# generate classification model from model_checkpoint
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, problem_type="single_label_classification")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
# display architecture
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

### preprocess data

In [25]:
# create tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

# add pad token if none exists
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [26]:
# create tokenize function
def tokenize(batch):

    #tokenize and truncate text
    tokens = tokenizer(batch['text'], padding=True, truncation=True, max_length=512)
    print(batch['label'])
    return tokens


# tokenize training and validation datasets
tokenized_dataset = dataset.map(tokenize, batched=True)

tokenized_dataset


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 546
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 116
    })
})

In [27]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### evaluation

In [28]:
# import accuracy evaluation metric
accuracy = evaluate.load("accuracy")

### Train model

In [29]:

peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=16,
                        lora_alpha=32,
                        lora_dropout=0.05,
                        target_modules=["q_lin", "k_lin", "v_lin"])

In [30]:
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='SEQ_CLS', inference_mode=False, r=16, target_modules={'v_lin', 'q_lin', 'k_lin'}, lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [31]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 1,034,498 || all params: 67,989,508 || trainable%: 1.5216


In [32]:
# hyperparameters
lr = 1e-3
batch_size = 2
num_epochs = 5

In [33]:
# define training arguments
training_args = TrainingArguments(
    output_dir= model_output,
    learning_rate=lr,
    auto_find_batch_size=True,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch"
)

In [34]:
# show metrics for f1 and auprc
from sklearn.metrics import f1_score, precision_recall_curve, auc

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')
    precision, recall, _ = precision_recall_curve(labels, preds)
    auprc = auc(recall, precision)
    return {
    'f1': f1,
    'auprc': auprc
    }

In [35]:
# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics
)

In [36]:
# train model
trainer.train()

  0%|          | 0/345 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 0.16370032727718353, 'eval_f1': 0.9222926277621211, 'eval_auprc': 0.958382129689872, 'eval_runtime': 0.7345, 'eval_samples_per_second': 157.921, 'eval_steps_per_second': 20.421, 'epoch': 1.0}


c:\Users\chefb\OneDrive\Laufbahn\Hochschule\S8\bachelorarbeit\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

...

  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 0.21127718687057495, 'eval_f1': 0.9568805340919284, 'eval_auprc': 0.9798850574712643, 'eval_runtime': 0.6823, 'eval_samples_per_second': 170.009, 'eval_steps_per_second': 21.984, 'epoch': 5.0}


c:\Users\chefb\OneDrive\Laufbahn\Hochschule\S8\bachelorarbeit\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'train_runtime': 94.076, 'train_samples_per_second': 29.019, 'train_steps_per_second': 3.667, 'train_loss': 0.09369468136110168, 'epoch': 5.0}


TrainOutput(global_step=345, training_loss=0.09369468136110168, metrics={'train_runtime': 94.076, 'train_samples_per_second': 29.019, 'train_steps_per_second': 3.667, 'total_flos': 370311877877760.0, 'train_loss': 0.09369468136110168, 'epoch': 5.0})

In [37]:
hf_name = 'cyrp' # your hf username or org name
model_id = hf_name + "/" + model_output # you can name the model whatever you want

In [38]:
model.push_to_hub(model_id) # save model

c:\Users\chefb\OneDrive\Laufbahn\Hochschule\S8\bachelorarbeit\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/4.14M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/cyrp/distilbert-base-uncased-deepset-promptinjection/commit/dac04ed21aa05f6b80dd75d5ce09f0cf2b20f474', commit_message='Upload model', commit_description='', oid='dac04ed21aa05f6b80dd75d5ce09f0cf2b20f474', pr_url=None, pr_revision=None, pr_num=None)

In [39]:
trainer.push_to_hub(model_output) # save trainer

events.out.tfevents.1727291028.DESKTOP-LM2NI5P.3944.0:   0%|          | 0.00/6.87k [00:00<?, ?B/s]

events.out.tfevents.1727293754.DESKTOP-LM2NI5P.28740.1:   0%|          | 0.00/6.87k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1727293462.DESKTOP-LM2NI5P.28740.0:   0%|          | 0.00/6.87k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/cyrp/distilbert-base-uncased-deepset-promptinjection/commit/25ce32b40f175f8e39fff0441607ed40606eaf58', commit_message='distilbert-base-uncased-deepset-promptinjection', commit_description='', oid='25ce32b40f175f8e39fff0441607ed40606eaf58', pr_url=None, pr_revision=None, pr_num=None)